# Sharpe Ratio

## Old

In [1]:
import pandas as pd
import yfinance as yf  # Yahoo Finance

from statsmodels.graphics import tsaplots
from statsmodels.tsa.statespace.sarimax import SARIMAX
from calendar import monthrange
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.dates as mdates

KeyboardInterrupt: 

In this notebook, we will be attempting to predict the sharpe ratio of several commodity futures. All data in this notebook was collected via Yahoo Finance's API. We will predict the sharpe ratio using the futures with the nearest expiration dates, specifically:

1. Corn Futures (ZC=F), daily close values.
2. Soybean Futures (ZS=F), daily close values.
3. Wheat Futures (ZW=F), daily close values.
4. Oats Futures (ZO=F), daily close values.
5. Rice Futures (ZR=F), daily close values.

We start by retrieving information of the risk-free stock, which in this case we chose to be ^IRX (13 WEEK USA TREASURY BILL). All values used will be CLOSE values. The data collected is of the last 10 years.

In [ ]:
rf = yf.Ticker('^IRX')
rf = rf.history(period='max')
rf['Datetime'] = rf.index
rf.reset_index(drop=True, inplace=True)
rf['Date'] = rf['Datetime'].dt.date
rf['Return'] = rf['Close'].pct_change()

In [ ]:
s = yf.Ticker('SB=F')
s.info['shortName']

Next, we retrieve data from the last 10 years of all commodity prices, those listed above.

In [ ]:
products = {'Corn': 'ZC',
            'Soybean': 'ZS',
            'Wheat': 'ZW',
            'Oats': 'ZO',
            'Rice': 'ZR',}

df_futures = pd.DataFrame()

for product in products:
    value = f'{products[product]}=F'
    futures = yf.Ticker(value)
    futures = futures.history(period='max')
    df_futures[product] = futures['Close']


df_futures['Datetime'] = df_futures.index
df_futures.reset_index(drop=True, inplace=True)
df_futures['Date'] = df_futures['Datetime'].dt.date

In [ ]:
df_futures.dropna()

In [ ]:
# get the dataframe values on the date 2018-01-23
df_futures[df_futures['Date'] == pd.to_datetime('2018-01-24').date()]

In [ ]:
df_futures = df_futures.merge(rf[['Date', 'Return']], on='Date', how='left')
df_futures.rename(columns={'Return': 'rf'}, inplace=True)
df_futures.dropna(inplace=True)

In [ ]:
# compute sharp ratio for each product in each year
sharp_ratios = {}

for product in products:
    df_futures[f'{product}_return'] = df_futures[product].pct_change()
    df_futures[f'{product}_excess_return'] = df_futures[f'{product}_return'] - df_futures['rf']
    # compute sharp ratio for each year
    sharp_ratios[product] = {}
    for year in range(2015, 2024):  # only data with full years
        df_year = df_futures[df_futures['Datetime'].dt.year == year]
        sharp_ratio = df_year[f'{product}_excess_return'].mean() / df_year[f'{product}_excess_return'].std()
        annualized_sharp_ratio = sharp_ratio * (252 ** 0.5)
        sharp_ratios[product][year] = annualized_sharp_ratio

In [ ]:
print('Yearly Sharpe Ratio For Each Product (Compared to Risk Free Rate based on 3 month treasury bill)')
for product in sharp_ratios:
    print(product)
    for year in sharp_ratios[product]:
        print(f'{year}: {sharp_ratios[product][year]}')
    print('\n')

Now, we will compute the sharpe ratio on a bi-weekly basis, making it so that we will have more sharpe ratios that we can train a model on.

In [ ]:
WINDOW_SIZE = 14

normalized_sharpe_ratios = {}

for product in products:
    df_futures[f'{product}_return'] = df_futures[product].pct_change()
    df_futures[f'{product}_excess_return'] = df_futures[f'{product}_return'] - df_futures['rf']
    rolling_mean = df_futures[f'{product}_excess_return'].rolling(window=WINDOW_SIZE, step=WINDOW_SIZE).mean()
    rolling_std = df_futures[f'{product}_excess_return'].rolling(window=WINDOW_SIZE, step=WINDOW_SIZE).std()
    normalized_sharpe_ratios[product] = rolling_mean / rolling_std * (WINDOW_SIZE ** 0.5)
    # add column of date
    normalized_sharpe_ratios[product]['Datetime'] = df_futures['Datetime']

# plot normalized sharp ratio
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


normalized_sharpe_ratios['Corn']

# # plot normalized sharp ratio in 4 subplots
# fig, axs = plt.subplots(5, 1, figsize=(10, 10))
# fig.suptitle('Normalized Sharpe Ratios for Corn, Soybean, Wheat, and Oats')
# for i, product in enumerate(products):
#     axs[i].plot(df_futures['Datetime'], normalized_sharpe_ratios[product], label=product)
#     axs[i].xaxis.set_major_locator(mdates.YearLocator())
#     axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
#     axs[i].set_title(product)
#     axs[i].legend()
#     axs[i].grid()
# plt.tight_layout()
# plt.show()


# (normalized_sharpe_ratios['Corn'].dropna() - normalized_sharpe_ratios['Soybean'].dropna()).mean()


## Using LSTMs to Predict Sharpe Ratio of the Next Quarter

In [2]:
import pandas as pd
import yfinance as yf  # Yahoo Finance

In [27]:
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from sklearn.decomposition import PCA

In [4]:
import gensim.downloader as api

# Load word vectors
word_vectors = api.load("glove-wiki-gigaword-100")


[==================================================] 100.0% 128.1/128.1MB downloaded


In [5]:
def get_vector(string):
    sol = np.zeros(100)
    for word in string.lower().split():
        try:
            sol += word_vectors[word]
        except:
            continue
    return sol

In [6]:
with open('commodities_tickers.json', 'r') as f:
    data = json.load(f)

categories = data.keys()
categories_vectors = {category: get_vector(category) for category in categories}

data_reshaped = {key2: [key1, value2] for key1, value1 in data.items() for key2, value2 in value1.items()}

# make matrix of word vectors for each commodity
commodities = data_reshaped.keys()
commodities_vectors = np.array([get_vector(commodity) for commodity in commodities])

# apply PCA to reduce the dimensionality of the word vectors
pca = PCA(n_components=5)
commodities_vectors_pca = pca.fit_transform(commodities_vectors)
categories_vectors_pca = pca.transform(np.array(list(categories_vectors.values())))
categories_vectors_pca = {key: value for key, value in zip(categories, categories_vectors_pca)}

data_reshaped = {key: [np.concatenate([categories_vectors_pca[value[0]], commodities_vectors_pca[i]]), value[1]] for i, (key, value) in enumerate(data_reshaped.items())}

In [37]:
RISK_FREE = yf.Ticker('^IRX')
RISK_FREE = RISK_FREE.history(period='max')
RISK_FREE['Datetime'] = RISK_FREE.index
RISK_FREE.reset_index(drop=True, inplace=True)
RISK_FREE['Date'] = RISK_FREE['Datetime'].dt.date
RISK_FREE['Returns-RF'] = RISK_FREE['Close'].pct_change()

def retrieve_feature(product, data_reshaped):
    return yf.Ticker(data_reshaped[product][1]).history(period='max')

def feature_extraction(data):
    data['Datetime'] = data.index
    data.reset_index(drop=True, inplace=True)
    data['Date'] = data['Datetime'].dt.date
    data.dropna(inplace=True)
    data['Return'] = data['Close'].pct_change()
    return data[['Datetime', 'Date', 'Return']]

def risk_free_return(data):
    data = data.merge(RISK_FREE[['Date', 'Returns-RF']], on='Date', how='left')
    data.rename(columns={'Returns-RF': 'rf'}, inplace=True)
    data.dropna(inplace=True)
    data['Risk-Adjusted Return'] = data['Return'] - data['rf']
    return data[['Datetime', 'Risk-Adjusted Return']]

def sharpe_ratio(data):
    # add feature of quarter such that jan/feb/mar -> 1, apr/may/jun -> 2, jul/aug/sep -> 3, oct/nov/dec -> 4
    data['Quarter'] = data['Datetime'].dt.quarter
    # group by quarter and year, compute the mean and standard deviation of risk-adjusted return
    data_grouped = data.groupby([data['Datetime'].dt.year, 'Quarter'])

    mean = data_grouped['Risk-Adjusted Return'].mean()
    std = data_grouped['Risk-Adjusted Return'].std()
    length = data_grouped['Risk-Adjusted Return'].count()

    sharp_ratio = mean / std * (length ** 0.5)
    return sharp_ratio.reset_index()

In [38]:
def get_final_series(product, data_reshaped):
    data = retrieve_feature(product, data_reshaped)
    data = feature_extraction(data)
    data = risk_free_return(data)
    sharpe = sharpe_ratio(data)
    values = sharpe.values
    # for every row in values, concatenate the corresponding vector in data_reshaped
    meta_data = data_reshaped[product][0]
    # copy meta_data for each row in values
    meta_data = np.tile(meta_data, (values.shape[0], 1))
    return np.concatenate([meta_data, values], axis=1)


In [39]:
data_train = [torch.tensor(get_final_series(product, data_reshaped)) for product in data_reshaped.keys() if product not in ['Corn', 'Soybeans', 'Wheat', 'Oats']]
data_test = {product: torch.tensor(get_final_series(product, data_reshaped)) for product in ['Corn', 'Soybeans', 'Wheat', 'Oats']}


In [40]:
class Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device):
        super(Predictor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.device = device

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

In [88]:
class Predictor3(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device):
        super(Predictor3, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(3, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size + 10, 8)
        self.fc2 = nn.Linear(8, output_size)
        self.device = device

    def forward(self, x):
        x_serial = x[:, :, -3:]
        # print(x_serial)
        x_static = x[:, 0, :-3]
        # print(x_static)

        out, (h_n, c_n) = self.lstm(x_serial)
        # print(torch.cat([out[:, -1, :], x_static], dim=1))
        out = F.relu(self.fc1(torch.cat([out[:, -1, :], x_static], dim=1)))
        return self.fc2(out)

In [42]:
class LSTMPredictor2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device):
        super(LSTMPredictor2, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.device = device

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        out, (h_n, c_n) = self.lstm(x, (h0, c0))
        out = self.fc(h_n)
        return out

In [96]:
def create_inputs(series, device):
    x = series.unsqueeze(0).float().to(device)
    x = x[:, :-1, :]
    y = series[-1, -1].unsqueeze(0).unsqueeze(0).float().to(device)
    return x, y


In [73]:
def create_inputs2(series, device):
    train_data_size = int(0.8 * series.shape[0])
    train_data = series[:train_data_size]
    test_data = series[train_data_size:-1]
    train_x = train_data.unsqueeze(0).float().to(device)
    val_x = test_data.unsqueeze(0).float().to(device)
    train_y = series[train_data_size, -1].unsqueeze(0).unsqueeze(0).float().to(device)
    val_y = series[-1, -1].unsqueeze(0).unsqueeze(0).float().to(device)
    return train_x, train_y, val_x, val_y

In [105]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Predictor3(13, 64, 2, 1, device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.MSELoss()

# A = torch.randn(1, 101, 13)
# A[:, :, :-1] = 0
# A = data_train[0].unsqueeze(0).float().to(device)
running_loss = np.inf
for epoch in range(1, 101):
    previous_running_loss = running_loss
    running_loss = 0
    for data in data_train:
        train_x, train_y = create_inputs(data, device)
        optimizer.zero_grad()
        outputs = model(train_x)

        loss = criterion(outputs, train_y)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    if abs(previous_running_loss - running_loss) < 1e-4:
      print(f'Epoch {epoch+1}, Loss {running_loss / len(data_train)}')
      break
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss {running_loss / len(data_train)}')
    # optimizer.zero_grad()
    # output = model(A[:, :-1, :])
    # loss = criterion(output, A[:, 1:, -1].unsqueeze(2))
    # loss.backward()
    # optimizer.step()
    # print(f'Epoch {epoch+1}, Loss {loss.item()}')


Epoch 10, Loss 0.6340332164196297
Epoch 20, Loss 0.5243942360772053
Epoch 30, Loss 0.45294591273086554
Epoch 40, Loss 0.4111247294153145
Epoch 50, Loss 0.3846084435066587
Epoch 60, Loss 0.3663525033320184
Epoch 70, Loss 0.352064409966988
Epoch 80, Loss 0.3418859649362275
Epoch 90, Loss 0.33388855189987227
Epoch 100, Loss 0.32638242420507596


In [ ]:
with torch.no_grad():
    for product in data_test:
        data = data_test[product]
        val_x, val_y = create_inputs(data, device)
        outputs = model(val_x)
        # compute absolute error
        error = (val_y - outputs) ** 2
        try:
            overall = torch.stack(overall, error)
        except:
            overall = error

print('Test MSE:', overall.mean().item())

In [ ]:
with torch.no_grad():
    for product in data_test:
        data = data_test[product]
        x = data.unsqueeze(0).float().to(device)
        outputs = model(x)
        print(f'Sharpe ratio prediction for {product} for next quarter: {outputs.item()}')